<a href="https://colab.research.google.com/github/amenem/test/blob/master/RNN_SelfAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install -q kaggle
! pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.47.0-cp36-none-any.whl size=17289 sha256=38d940a981d56d737251591afa1c8e3d819c5967efea74015e4c154e6a5ad5e2
  Stored in directory: /root/.cache/pip/wheels/70/87/01/76c703d5401b65e323927c1fdc665f3fb143282ff67d71e859
Successfully built keras-self-attention


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
import warnings  
warnings.filterwarnings('ignore')    

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amenem","key":"f0913597f85de0138806c77add36fa0f"}'}

In [3]:
import os
os.chdir('/content/drive/My Drive')



In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c fake-news

  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 5.68MB/s]
 53% 5.00M/9.42M [00:00<00:00, 11.9MB/s]
100% 9.42M/9.42M [00:00<00:00, 19.2MB/s]
 81% 30.0M/37.0M [00:01<00:00, 11.1MB/s]
100% 37.0M/37.0M [00:01<00:00, 20.8MB/s]


In [ ]:
! unzip ./train.csv.zip -d ./

Archive:  ./train.csv.zip
  inflating: ./train.csv             


In [ ]:
! unzip ./test.csv.zip -d ./

Archive:  ./test.csv.zip
  inflating: ./test.csv              


In [4]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [41]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras_self_attention import SeqSelfAttention

In [8]:
df = pd.read_csv('./train.csv')

In [9]:
df=df.dropna(subset=['text'])

In [10]:
df.shape

(20761, 5)

In [11]:
df['label'].value_counts()

0    10387
1    10374
Name: label, dtype: int64

In [12]:
## looks balanced data


In [27]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
    
STOP_WORDS = set(stopwords.words('english')) 
def filter_sw(text):
  text = re.sub('[^a-zA-Z]',' ',text)
  text = text.lower()
  tokens = word_tokenize(text)
  filtered = [token for token in tokens if token not in STOP_WORDS]
  return filtered

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Remove stopwords

In [28]:
df['text_ws']=df['text'].apply(filter_sw)

In [22]:
X = df['text_ws'].values
y=df['label']
max_num_words = 20000
tokenizer = Tokenizer(max_num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

In [29]:
X

array([list(['house', 'dem', 'aide', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweeted', 'darrell', 'lucus', 'october', 'subscribe', 'jason', 'chaffetz', 'stump', 'american', 'fork', 'utah', 'image', 'courtesy', 'michael', 'jolley', 'available', 'creative', 'commons', 'license', 'apologies', 'keith', 'olbermann', 'doubt', 'worst', 'person', 'world', 'week', 'fbi', 'director', 'james', 'comey', 'according', 'house', 'democratic', 'aide', 'looks', 'like', 'also', 'know', 'second', 'worst', 'person', 'well', 'turns', 'comey', 'sent', 'infamous', 'letter', 'announcing', 'fbi', 'looking', 'emails', 'may', 'related', 'hillary', 'clinton', 'email', 'server', 'ranking', 'democrats', 'relevant', 'committees', 'hear', 'comey', 'found', 'via', 'tweet', 'one', 'republican', 'committee', 'chairmen', 'know', 'comey', 'notified', 'republican', 'chairmen', 'democratic', 'ranking', 'members', 'house', 'intelligence', 'judiciary', 'oversight', 'committees', 'agency', 'reviewing', 'emails',

In [30]:
sequences = tokenizer.texts_to_sequences(X)
max_length=200
sequences_pad = pad_sequences(sequences,maxlen=max_length,padding='post')

In [ ]:
### Embedding

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [31]:
glove_file = 'glove.6B.100d.txt'
print('Indexing word vectors.')

embeddings_index = {}
f = open(glove_file, encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [32]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [33]:
X_train, X_val, y_train, y_val = train_test_split(sequences_pad,y,test_size=.3, random_state=23)

In [34]:
EMBED_DIM=100
embedding_matrix=np.zeros((len(word_index)+1, EMBED_DIM))
for word,idx in word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[idx]=embedding_vector

In [35]:
embedding_layer = Embedding(len(word_index)+1, EMBED_DIM, input_length=max_length, 
                            weights=[embedding_matrix],
                            trainable=False)

In [36]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          22163600  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 256)          234496    
_________________________________________________________________
seq_self_attention (SeqSelfA (None, None, 256)         16449     
_________________________________________________________________
dense (Dense)                (None, None, 1)           257       
Total params: 22,414,802
Trainable params: 251,202
Non-trainable params: 22,163,600
_________________________________________________________________


In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
checkpoint = ModelCheckpoint('RNN_selfAttention.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto') 
history=model.fit(X_train,y_train,epochs=20,validation_data=(X_val,y_val),callbacks=[checkpoint])

Epoch 1/20
455/455 [==============================] - 648s 1s/step - loss: 0.3708 - accuracy: 0.8352 - val_loss: 0.2869 - val_accuracy: 0.8810
Epoch 2/20
455/455 [==============================] - 646s 1s/step - loss: 0.2565 - accuracy: 0.8908 - val_loss: 0.2351 - val_accuracy: 0.9051
Epoch 3/20
455/455 [==============================] - 648s 1s/step - loss: 0.2141 - accuracy: 0.9112 - val_loss: 0.2359 - val_accuracy: 0.9016
Epoch 4/20
455/455 [==============================] - 647s 1s/step - loss: 0.2052 - accuracy: 0.9149 - val_loss: 0.2480 - val_accuracy: 0.8963
Epoch 5/20
455/455 [==============================] - 644s 1s/step - loss: 0.1855 - accuracy: 0.9244 - val_loss: 0.1964 - val_accuracy: 0.9227
Epoch 6/20
455/455 [==============================] - 645s 1s/step - loss: 0.1589 - accuracy: 0.9371 - val_loss: 0.1880 - val_accuracy: 0.9304
Epoch 7/20
455/455 [==============================] - 648s 1s/step - loss: 0.1450 - accuracy: 0.9414 - val_loss: 0.1937 - val_accuracy: 0.9220

KeyboardInterrupt: ignored